In [1]:
import numpy as np
import pandas as pd
import numpy as np
import os
import time
from google.cloud import bigquery
import pandas_gbq
import glob

In [5]:
client = bigquery.Client()

In [12]:
query = "select * from `repositoriodedadosgpsp.percepcao_mensuracao_qualidade_estado.INEP_esforco_docente_temporal` order by ano"  

In [13]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')

Downloading:  55%|█████▌    |

In [ ]:
os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\Tabelas para o usuário baixar")

In [ ]:
df.to_csv("INEP_esforco_docente_temporal.csv",index=False, encoding="latin-1", sep=';', decimal=',')

-----------------------------------------------------------

Testes

In [ ]:
df = pd.read_csv('INEP_esforco_docente_temporal.csv',encoding="latin-1", sep=';', decimal=',')
df

,ano,sigla_uf,docente_grupos,ano_escolar,localizacao,dependencia_adm,percent
0,2013,SE,Grupo 1,Ensino Médio,Rural,Municipal,NaN
1,2013,PI,Grupo 4,Ensino Fundamental - Todos os anos,Rural,Municipal,33.3
2,2013,PE,Grupo 4,Ensino Médio,Rural,Municipal,NaN
3,2013,PI,Grupo 5,Ensino Fundamental - Todos os anos,Rural,Municipal,0.0
4,2013,SP,Grupo 4,Ensino Fundamental - Anos Finais,Urbana,Estadual,47.8
...,...,...,...,...,...,...,...
33222643,2022,AM,Grupo 4,Ensino Médio,Rural,Municipal,NaN
33222644,2022,CE,Grupo 4,Ensino Fundamental - Todos os anos,Rural,Municipal,62.5
33222645,2022,MG,Grupo 3,Ensino Médio,Urbana,Estadual,26.1
33222646,2022,PB,Grupo 5,Ensino Médio,Urbana,Privada,NaN


-----------------------------------------------------------

Automatizando troca de csv

In [7]:
os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\Tabelas para o usuário baixar")

In [ ]:
## baixando os atuais com , e exportando com ";"

In [15]:
## listando todos os itens do diretório
len(glob.glob('*.csv'))

42

In [17]:
## Serve apenas na primeira vez . 

for i in list(range(0,len(glob.glob('*.csv')))):
    
    df= pd.read_csv(glob.glob('*.csv')[i])
    df.to_csv(glob.glob('*.csv')[i],index=False, encoding="latin-1", sep=';', decimal=',')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe1 in position 139: invalid continuation byte